# Imports

In [ ]:
import pandas as pd
import plotly.express as px
import geopandas as gpd

# Importation des données

In [ ]:
france = pd.read_csv("datas/caracteristics_complete.csv")

# Carte de densités des accidents
La carte de densité permet d'identifier les zones les plus denses en accidents, ici l'on peut voir un analysant la carte que les zones les plus denses sous bel et bien les grandes villes comme Paris. Et si l'on zoome suffisamment on peut identifier que les zones étant les plus propices aux accidents sont les intersections.

In [ ]:
def density_map(df: pd.DataFrame, params: dict = dict()):
    """
    Create a density map (heat map)
    """
    density_params = dict(
        lat="lat",
        lon="long",
        radius=5,
        mapbox_style="open-street-map",
        zoom=5,
        height=1000
    )

    fig = px.density_mapbox(
        df,
        **density_params,
        **params
    )

    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.show(renderer="notebook")

In [ ]:
density_map(france)

# Carte de tous les accidents
Cette carte permet d'identifier individuellement les accidents de notre dataframe, grâce à cette carte on peut identifier les anomalies où les accidents ne se sont pas déroulé en France, mais qui sont pourtant dans notre jeu de données.

In [ ]:
def scatter_map(df: pd.DataFrame, params: dict = dict()):
    """
    Create a scatter map (each dot represent an object)
    """
    scatter_params = dict(
        lat="lat",
        lon="long",
        hover_data=["an", "mois", "jour", "adr", "dep", "Num_Acc", "cc", "name", "admin1", "admin2"],
        color_discrete_sequence=["fuchsia"],
        mapbox_style="open-street-map",
        zoom=5,
        height=1000
    )

    fig = px.scatter_mapbox(
        df,
        **scatter_params,
        **params
    )

    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.show(renderer="notebook")

In [ ]:
scatter_map(france)

# Carte des départements
Cette représentation est intéressante afin d'isoler les densités d'une certaine quantité par départements. Ici l'on affiche la densité d'accident par département, qui est représenté par une couleur allant du bleu au jaune, respectivement pour une petite et grande densité d'accidents.

In [ ]:
def dep_map(df, params=dict()):
    """
    Create a France departement map, with a color that represents a density
    """
    fr = gpd.read_file("datas/departements.geojson")

    choropleth_params = dict(
        geojson=fr,
        locations="dep",
        color="count",
        mapbox_style="carto-positron",
        featureidkey="properties.dep",
        zoom=5,
        center={"lat": 46, "lon": 2},
        opacity=0.5,
        labels={"value": "Nombre d'accidents"},
        height=1000
    )

    fig = px.choropleth_mapbox(
        df,
        **choropleth_params,
        **params
    )

    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show(renderer="notebook")

In [ ]:
dep_count = france.dep.value_counts().reset_index()
dep_count.columns = ["dep", "count"]
dep_count.dep /= 10

dep_map(dep_count)

# Fonction créatrice des graphiques d'évolution
Il serait maintenant intéressant de représenter une évolution de ces cartes en fonction d'un paramètre, les fonctions suivantes génèrent une map munie d'un slider afin de connaître cette évolution.

In [ ]:
def dep_evolution(field: str):
    """
    Create a France departement map, with a variable value that depends on a field
    """
    df_accident_month_dep = france.groupby([field, "dep"]).count()["Num_Acc"].reset_index()
    df_accident_month_dep.columns = [field, "dep", "count"]
    df_accident_month_dep.dep /= 10
    
    dep_map(
        df_accident_month_dep,
        dict(animation_frame=field)
    )

def scatter_evolution(field: str):
    """
    Create scatter map, with a variable value that depends on a field
    """
    scatter_map(france, dict(animation_frame=field))

def density_evolution(field: str):
    """
    Create density map, with a variable value that depends on a field
    """
    density_map(france, dict(animation_frame=field))

def pie(df: pd.DataFrame, column: str, title: str = None):
    """
    Create a pie chart from a specific column of a dataframe
    """
    data = df[column].value_counts() / len(df)
    
    fig = px.pie(
        data,
        title=column if title == None else title,
        values=data,
        names=data.index
    )

    fig.update_traces(textposition="inside", textinfo="percent+label")

    return fig.show()

# Évolution de la carte en fonction des mois
Il est important d'également connaître la répartition des accidents pour chacun des mois, afin de bien interpréter certains graphiques de map, comme la représentation par département qui ne nous permet pas de savoir la répartition des accidents par mois. Cela est moins le cas pour une map de type "scatter", car on identifie la répartition des mois en comptant le nombre de points (ce qui se fait de manière plutôt implicite dans notre cerveau). Sur les cartes on peut voir qu’en fonction de mois, la variation des zones où les accidents sont les plus propices est faible. On remarque tout de même, sur la carte représentant la densité des accidents, que lors de l'été il y'a plus d'accidents dans le sud que durant le reste de l'année.

In [ ]:
pie(france, "mois", "Répartition des mois")

In [ ]:
dep_evolution("mois")

In [ ]:
scatter_evolution("mois")

In [ ]:
density_evolution("mois")

# Évolution de la carte durant les années
Pour les années on voit que la répartition est moins homogène que pour les mois, cela reste tout de même correct, on peut cependant noter qu’entre 2016 et 2006 il y'a une très grande différence du nombre d'accidents auxquels on a accès. On peut aussi voir que nous n'avons pas d'accidents pour 2009. Sur les cartes, on remarque que très peu de changement, car les populations n'ont pas énormément migré, cela aurait pu être le cas si des événements extraordinaires s’étaient produits durant ces années, comme une guerre, un changement de capitale, etc. ça pourrait être également le cas pour un pays en voie de développement.

In [ ]:
pie(france, "an", "Répartition des années")

In [ ]:
dep_evolution("an")

In [ ]:
scatter_evolution("an")

In [ ]:
density_evolution("an")

# Carte en fonction des conditions lumineuses
La carte affichant les accidents survenus en plein jour n'est pas réellement intéressante, cependant pour ceux qui se sont passés la nuit nous permet d'identifier les zones où l'éclairage n'est pas optimal.

In [ ]:
pie(france, "lum_str", "Répartition des conditions lumineuses")

In [ ]:
density_evolution("lum_str")

In [ ]:
scatter_evolution("lum_str")